In [1]:
import os
from dotenv import load_dotenv
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from dotenv import load_dotenv

from vectaraIntegration import query, get_knowledge, initialize_vectara, upload_all_file


In [3]:
get_knowledge

<function vectaraIntegration.get_knowledge(user_query, vectorstore)>

In [2]:
vectara_client = initialize_vectara()
knowledge_content, summary = get_knowledge("Hi There", vectara_client)

In [5]:
knowledge_content

''

In [4]:
summary

'The returned results did not contain sufficient information to be summarized into a useful answer for your query. Please try a different search or restate your query differently.\n'

In [37]:
# Selecting LLM Model from HuggingFace 
REPO_ID = "google/gemma-2b-it"

In [38]:
# Getting HF_TOKEN
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")

Run Locally using LangChain Pipeline

In [54]:
# hf = HuggingFacePipeline.from_model_id(
#     model_id=REPO_ID,
#     task="text-generation",
#     pipeline_kwargs={"max_new_tokens": 100},
# )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device has 1 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.


Run Model using  API Inference HuggingFace

In [46]:
hf = HuggingFaceHub(
    huggingfacehub_api_token = HF_TOKEN,
    repo_id=REPO_ID,
    task="text-generation",
    model_kwargs={
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

# llm = HuggingFaceEndpoint(
#     repo_id=REPO_ID,
#     repetition_penalty= 1.03, 
#     temperature=0.5, 
#     max_new_tokens=200, #Maximum number of generated tokens
#     huggingfacehub_api_token=HF_TOKEN
# )



In [47]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [49]:
llm_chain = LLMChain(prompt=prompt, llm=hf) #ini legacy ya.. nanti cari LCEL CHAINS yang baru

Getting The Data

In [1]:
from langchain_community.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
# pip install unstructured

In [37]:
txtDatas = []
for filename in os.listdir("../Bank Product Data/"):
    if(filename.endswith(".txt")):
        txtDatas.append(filename)

print(txtDatas)

['Business bank accounts.txt', 'Buy now, pay later.txt', 'Car Loans.txt', 'Commonwealth Bank of Australia.txt', 'CommonWealth Our company.txt', 'Commonwealth Private.txt', 'Credit cards.txt', 'Debt consolidation loans.txt', 'Disputing a transaction.txt', 'electric-vehicle-loan.txt', 'Everyday Account Smart Access.txt', 'EVERYDAY ACCOUNT.txt', 'Fixed Rate Personal Loans.txt', 'Foreign Currency Account.txt', 'GoalSaver.txt', 'Helpful ways to offset the rising cost of living.txt', 'Home Improvement Loan.txt', 'How do I activate my CommBank card.txt', 'How do I close my CommBank account.txt', 'How do I report my card lost, stolen or damaged.txt', 'How long does it take to transfer money.txt', 'InstalPay.txt', 'International Money Transfers (IMT).txt', 'International Money Transfers.txt', 'Klarna.txt', 'NetBank Saver.txt', 'Pensioner Security Account.txt', 'Personal loans.txt', 'Premier and Private Banking.txt', 'Premier Banking.txt', 'Saving Account & Term Deposit.txt', 'Secured Personal L

In [42]:
output_filename = "combined.txt"

def concat_text_files(filenames, output_filename):
  with open(output_filename, 'w') as outfile:
    for filename in filenames:
      path = "../Bank Product Data/"+filename
      with open(path, 'r', encoding='utf-8') as infile:
        outfile.write(infile.read())

In [43]:
concat_text_files(txtDatas, output_filename)

UnicodeEncodeError: 'charmap' codec can't encode character '\u202f' in position 5018: character maps to <undefined>

In [48]:
loader = TextLoader("./combined.txt")
data = loader.load()

In [ ]:
# loader = WebBaseLoader("https://www.ocbc.com.hk/webpages/en-us/html/retail_banking/personal/faq.html")


# loader = DirectoryLoader('../', glob="**/*.txt")
# docs = loader.load()
# data = loader.load()
# print(len(data))

# loader = DirectoryLoader("../Bank Product Data",{
#     ".txt": (path) => TextLoader(path),
# }
# )
# loader.load()

In [49]:
# split document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)


Created a chunk of size 2042, which is longer than the specified 1000
Created a chunk of size 1659, which is longer than the specified 1000


In [50]:
# create the open-source embedding function
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [57]:
# load it into Chroma
db = Chroma.from_documents(docs, embedding_function)

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [42]:
# Creating RAG Chain

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.vectorstores import Vectara

In [43]:
VECTARA_CUSTOMER_ID = os.getenv("VECTARA_CUSTOMER_ID")
VECTARA_CORPUS_ID = os.getenv("VECTARA_CORPUS_ID")
VECTARA_API_KEY = os.getenv("VECTARA_API_KEY")

def initialize_vectara():
    vectara = Vectara(
                    vectara_customer_id=VECTARA_CUSTOMER_ID,
                    vectara_corpus_id=VECTARA_CORPUS_ID,
                    vectara_api_key=VECTARA_API_KEY
                )
    return vectara

In [44]:
def get_knowledge_content(vectara, query, threshold=0.5):
    found_docs = vectara.similarity_search_with_score(
        query,
        score_threshold=threshold,
    )
    knowledge_content = ""
    for number, (score, doc) in enumerate(found_docs):
        knowledge_content += f"Document {number}: {found_docs[number][0].page_content}\n"
    return knowledge_content


In [50]:
# retriever = db.as_retriever(search_type="mmr", search_kwargs={'k': 4, 'fetch_k': 20})

# prompt = hub.pull("rlm/rag-prompt")

vectara_client = initialize_vectara()
vectara_retriever = vectara_client.as_retriever()

# https://blog.langchain.dev/langchain-vectara-better-together/
qa = RetrievalQA.from_llm(llm=llm_chain, retriever=vectara_retriever)
qa.run("what is commonwealth bank?")
# print(qa({“query”:  “According to the document, what did Vladimir Putin miscalculate?”}))

# prompt2 = PromptTemplate.from_template(
#     """You are a professional and friendly Bank Customer Service and you are helping a client with bank knowledge. Just explain him in detail the answer and nothing else. This is the issue: {issue} 
#     To assist him with his issue, you need to know the following information: {knowledge} 
#     """
# )

# user_input = "Why Choose CommBank for your next car?"

# knowledge_content = get_knowledge_content(vectara_client, user_input)

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# rag_chain = (
#     {knowledge_content, user_input}
#     | prompt2
#     | llm_chain
# )
# rag_chain = (prompt2 | llm_chain | StrOutputParser())

BadRequestError:  (Request ID: EG2b4-L5VbQTBwA7M6y76)

Bad request:
Input length of input_ids is 1180, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

In [33]:
# Ask the LLM
# runnable = prompt 
# | ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], openai_api_key=OPENAI_API_KEY) 
# | StrOutputParser()

rag_chain.invoke("Why Choose CommBank for your next car?")
# rag_chain.invoke({"knowledge": knowledge_content, "issue": user_input})

BadRequestError:  (Request ID: ZkaErUqXyZSbi5j6VP__S)

Bad request:
The following `model_kwargs` are not used by the model: ['stop_sequences', 'watermark', 'stop'] (note: typos in the generate arguments will also show up in this list)